<a href="https://colab.research.google.com/github/leolaipelt/EarthEnginePythonBasico/blob/master/8_Operacoes_com_Reducer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Operações com Reducer

Reducers possuem como principal característica obter através de uma entrada de uma coleção de imagens, por exemplo, e obter uma única saída. Usando essa operação, é possível agregar informações através do tempo, espaço e bandas.

- **Tempo**  = `imageCollection.reduce()`
- **Espaço** = `image.reduceRegion()` e `image.reduceNeighborhood()`
- **Bandas**  = `image.reduce()`,

[Retirado de: EEwPython](https://colab.research.google.com/github/csaybar/EEwPython/blob/master/index.ipynb)

Com o Reducer podemos obter informações estatísticas como: média, mediana, mínimo, máximo, de acordo com o parâmetro desejado. 

Um pouco complicado, não achou? Vamos ver alguns exemplos!

In [0]:
!pip install earthengine-api

In [0]:
import ee; ee.Authenticate()

In [0]:
ee.Initialize()

In [15]:
image = (ee.Image('LANDSAT/LC08/C01/T1/LC08_044034_20140318')
    .select(['B4', 'B3', 'B2']));
geometry=image.geometry()

#Reducer Espacial
#Calculando a Média das Bandas
meanDictionary = image.reduceRegion(
  reducer= ee.Reducer.mean(),
  geometry=geometry ,
  scale= 30,
  maxPixels= 1e9
);
print(meanDictionary.getInfo())

{'B2': 8544.545293463992, 'B3': 7730.444736292572, 'B4': 7176.731994489558}


In [0]:
#Reducer Temporal
collection = (ee.ImageCollection('LANDSAT/LT05/C01/T1_SR')
  .filterDate('2006-01-01', '2007-12-31')
  .filter(ee.Filter.eq('WRS_PATH', 223))
  .filter(ee.Filter.eq('WRS_ROW', 67)))

#Fazer a mediana de todas as Bandas presentes na coleção de imagens.
median = collection.reduce(ee.Reducer.median());

#Vale destacar que após realizar esse tipo de operação, as bandas são renomeadas automáticamente com o sufixo da ação realizada.
vis_param = {'bands': ['B3_median', 'B2_median', 'B1_median'],'min':0,'max':3000, 'gamma': 1.6};



In [0]:
#Vamos visualizar esse produto com o mapa que já criamos!
import folium

def add_ee_layer(self,imagem,visParams, nome):
  mapID = ee.Image(imagem).getMapId(visParams)
  folium.raster_layers.TileLayer(
    tiles ="https://earthengine.googleapis.com/map/"+mapID['mapid']+
      "/{z}/{x}/{y}?token="+mapID['token'],
    attr = "Nosso Mapinha do Curso de Python",
    name = nome,
    overlay = True,
    control = True
  ).add_to(self)
  
meuMap = folium.Map(location=[20, 0], zoom_start=3, height=800)

folium.Map.add_ee_layer = add_ee_layer

In [38]:
# Adicionando no mapa a imagem que criamos
meuMap.add_ee_layer(median, vis_param,'Mediana')

# Adicionando um controlador de layers no nosso mapa
meuMap.add_child(folium.LayerControl())

#Ver nosso mapa!
display(meuMap)